# Fundamentals of Deep Learning, Data Mining Minor

## Homework 1. Introduction to PyTorch. Complete neural networks.

### About the task.

In this problem you will predict the year of release of a song (**regression problem**) based on some sound features: [data](https://archive.ics.uci.edu/ml/datasets/yearpredictionmsd). In the cells below is the code to load the data. Note that the training and test samples are in the same file, so do NOT change the cell where the data is divided.

In [ ]:
import torch
from torch import nn
import pandas as pd
import numpy as np
import random

In [ ]:
#%conda install wget

ValueError: ignored

In [ ]:
!wget -O data.txt.zip https://archive.ics.uci.edu/ml/machine-learning-databases/00203/YearPredictionMSD.txt.zip

--2022-10-04 15:17:58--  https://archive.ics.uci.edu/ml/machine-learning-databases/00203/YearPredictionMSD.txt.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 211011981 (201M) [application/x-httpd-php]
Saving to: ‘data.txt.zip’

data.txt.zip        100%[===================>] 201.24M  66.0MB/s    in 3.0s    

2022-10-04 15:18:01 (66.0 MB/s) - ‘data.txt.zip’ saved [211011981/211011981]



In [ ]:
df = pd.read_csv('data.txt.zip', header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,81,82,83,84,85,86,87,88,89,90
0,2001,49.94357,21.47114,73.07750,8.74861,-17.40628,-13.09905,-25.01202,-12.23257,7.83089,...,13.01620,-54.40548,58.99367,15.37344,1.11144,-23.08793,68.40795,-1.82223,-27.46348,2.26327
1,2001,48.73215,18.42930,70.32679,12.94636,-10.32437,-24.83777,8.76630,-0.92019,18.76548,...,5.66812,-19.68073,33.04964,42.87836,-9.90378,-32.22788,70.49388,12.04941,58.43453,26.92061
2,2001,50.95714,31.85602,55.81851,13.41693,-6.57898,-18.54940,-3.27872,-2.35035,16.07017,...,3.03800,26.05866,-50.92779,10.93792,-0.07568,43.20130,-115.00698,-0.05859,39.67068,-0.66345
3,2001,48.24750,-1.89837,36.29772,2.58776,0.97170,-26.21683,5.05097,-10.34124,3.55005,...,34.57337,-171.70734,-16.96705,-46.67617,-12.51516,82.58061,-72.08993,9.90558,199.62971,18.85382
4,2001,50.97020,42.20998,67.09964,8.46791,-15.85279,-16.81409,-12.48207,-9.37636,12.63699,...,9.92661,-55.95724,64.92712,-17.72522,-1.49237,-7.50035,51.76631,7.88713,55.66926,28.74903


In [ ]:
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

train_size = 463715
X_train = X[:train_size, :]
y_train = y[:train_size]
X_test = X[train_size:, :]
y_test = y[train_size:]

In [ ]:
#to begin with, normalize the data x
from sklearn.preprocessing import StandardScaler
normalise = StandardScaler()
X_train = normalise.fit_transform(X_train)
X_test = normalise.transform(X_test)
X_test

array([[ 0.33829038, -0.62020406,  0.65605739, ..., -0.62364135,
         0.09026399,  0.26535103],
       [ 1.52861474, -0.08040993,  1.00107112, ..., -0.63468633,
        -0.47987357, -0.29659618],
       [ 0.38769709,  0.20850817,  0.06748658, ..., -1.15383427,
         0.03794659, -0.32255083],
       ...,
       [ 0.28672381,  0.22080478, -1.34327979, ..., -0.78948349,
         0.18515079, -0.42716277],
       [ 0.12841505,  0.60299416, -0.34028082, ...,  1.19191033,
        -0.0265535 ,  1.74303022],
       [ 1.39358232,  1.12093836,  0.50317056, ..., -0.74770053,
         0.06825626,  0.49217542]])

## Assignment 0. (0 points, but if not completed the maximum for the entire assignment &mdash; 0 points)

We will use RMSE as a quality metric. For the very first baseline, train a `Ridge` regression from `sklearn'. In addition, calculate the quality at the best constant prediction.

In [ ]:
from sklearn.linear_model import Ridge
model = Ridge()
x_2 = X_train
y_2 = y_train
model.fit(x_2, y_2)
y_pred = model.predict(X_test)
y_pred

array([1994.88490642, 2002.62689356, 2000.91120954, ..., 1996.98608934,
       2001.89238117, 1999.71164066])

In [ ]:
from sklearn.metrics import mean_squared_error
(mean_squared_error(y_test, y_pred))**(1/2)

9.510160820470437

In [ ]:
#Consider RMSE for the best constant prediction
years = np.unique(y)
rmses = []
ye = []
for i in years:
  y_test_2 = np.repeat(i, y_test.shape)
  rmses.append(mean_squared_error(y_test, y_test_2)**(1/2))
  ye.append(i)

minn = min(rmses)
year_const = ye[rmses.index(minn)]
print('RMSE:', minn, 'Year:', year_const)
#it turns out that the RMSE for constant 1998 is the smallest

RMSE: 10.863228020678134 Год: 1998


## Assignment 1. (maximum 10 points)

There are a few rules you need to adhere to:

- All learning pipelines must be written in PyTorch. You may use other libraries (`numpy`, `sklearn`, etc.), but only for data processing. That is, you can transform data and read metrics with these libraries as you wish, but you can't import models from `sklearn` and use them to get the quality you want &mdash; you can't. Also, you cannot use libraries for which PyTorch itself is a dependency.

- We don't restrict your choice of model architecture, but a full-coherent neural network will most likely be enough for you.

- You may not use any data other than the training sample for training.

- Model assembling is forbidden.

**MUST**  
draw graphs of loss/metric dependence on the training and test samples depending on the training time. If training takes a relatively small number of epochs, it is better to draw the dependence on the training step number, but if there are more epochs, then draw the dependence on epochs. If the checker does not see such a graph for your best model, he has the right to reduce the points for the task.

Your solution must be reproducible. If it is not, the reviewer has the right to lower your score for the assignment. To fix a random seed, use the function in the box below.



In [ ]:
!wget -O data.txt.zip https://archive.ics.uci.edu/ml/machine-learning-databases/00203/YearPredictionMSD.txt.zip

--2022-10-04 17:15:45--  https://archive.ics.uci.edu/ml/machine-learning-databases/00203/YearPredictionMSD.txt.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 211011981 (201M) [application/x-httpd-php]
Saving to: ‘data.txt.zip’

data.txt.zip        100%[===================>] 201.24M  22.9MB/s    in 9.8s    

2022-10-04 17:15:55 (20.6 MB/s) - ‘data.txt.zip’ saved [211011981/211011981]



In [ ]:
df = pd.read_csv('data.txt.zip', header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,81,82,83,84,85,86,87,88,89,90
0,2001,49.94357,21.47114,73.07750,8.74861,-17.40628,-13.09905,-25.01202,-12.23257,7.83089,...,13.01620,-54.40548,58.99367,15.37344,1.11144,-23.08793,68.40795,-1.82223,-27.46348,2.26327
1,2001,48.73215,18.42930,70.32679,12.94636,-10.32437,-24.83777,8.76630,-0.92019,18.76548,...,5.66812,-19.68073,33.04964,42.87836,-9.90378,-32.22788,70.49388,12.04941,58.43453,26.92061
2,2001,50.95714,31.85602,55.81851,13.41693,-6.57898,-18.54940,-3.27872,-2.35035,16.07017,...,3.03800,26.05866,-50.92779,10.93792,-0.07568,43.20130,-115.00698,-0.05859,39.67068,-0.66345
3,2001,48.24750,-1.89837,36.29772,2.58776,0.97170,-26.21683,5.05097,-10.34124,3.55005,...,34.57337,-171.70734,-16.96705,-46.67617,-12.51516,82.58061,-72.08993,9.90558,199.62971,18.85382
4,2001,50.97020,42.20998,67.09964,8.46791,-15.85279,-16.81409,-12.48207,-9.37636,12.63699,...,9.92661,-55.95724,64.92712,-17.72522,-1.49237,-7.50035,51.76631,7.88713,55.66926,28.74903


In [ ]:
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

train_size = 463715
X_train = X[:train_size, :]
y_train = y[:train_size]
X_test = X[train_size:, :]
y_test = y[train_size:]

In [ ]:
#нормализуем данные x для начала
from sklearn.preprocessing import StandardScaler
normalise = StandardScaler()
X_train = normalise.fit_transform(X_train)
X_test = normalise.transform(X_test)
X_test

array([[ 0.33829038, -0.62020406,  0.65605739, ..., -0.62364135,
         0.09026399,  0.26535103],
       [ 1.52861474, -0.08040993,  1.00107112, ..., -0.63468633,
        -0.47987357, -0.29659618],
       [ 0.38769709,  0.20850817,  0.06748658, ..., -1.15383427,
         0.03794659, -0.32255083],
       ...,
       [ 0.28672381,  0.22080478, -1.34327979, ..., -0.78948349,
         0.18515079, -0.42716277],
       [ 0.12841505,  0.60299416, -0.34028082, ...,  1.19191033,
        -0.0265535 ,  1.74303022],
       [ 1.39358232,  1.12093836,  0.50317056, ..., -0.74770053,
         0.06825626,  0.49217542]])

In [ ]:
def set_random_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
set_random_seed(200)

In [ ]:
#normalize the targeting
from sklearn.preprocessing import StandardScaler
normalise = StandardScaler()
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)
y_train = normalise.fit_transform(y_train)
y_test = normalise.transform(y_test)
y_test

array([[0.78739469],
       [0.42175579],
       [0.60457524],
       ...,
       [0.69598496],
       [0.69598496],
       [0.60457524]])

In [ ]:
#As in the first seminar, I create a dataset, a source: https://github.com/hse-ds/iad-deep-learning/blob/master/2022/seminars/sem01/sem01.ipynb
class MyDataset(torch.utils.data.Dataset):
    """
    Our dataset
    """
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return {"sample": torch.tensor(self.x[idx, :], dtype=torch.float), "target": self.y[idx]}

In [ ]:
model_2 = nn.Sequential()                   # create an empty model, in which we will add layers
model_2.add_module("start neural", nn.Linear(90, 50))  # added a layer with 90 neurons per input, because 90 features
model_2.add_module("l2", nn.ReLU())         # added an activation function
model_2.add_module("Dropout", nn.Dropout()) #I added a dropout to avoid overtraining
model_2.add_module("finish", nn.Linear(50, 1))  # added a layer
# model_2.add_module("l2", nn.ReLU())         # added an activation function
# model_2.add_module("finish", nn.Linear(20, 1))

In [ ]:
!pip install wandb --upgrade --quiet
import wandb

     |████████████████████████████████| 1.8 MB 2.1 MB/s 
     |████████████████████████████████| 162 kB 58.4 MB/s 
     |████████████████████████████████| 181 kB 60.7 MB/s 
     |████████████████████████████████| 63 kB 823 kB/s 
     |████████████████████████████████| 162 kB 58.8 MB/s 
     |████████████████████████████████| 158 kB 56.8 MB/s 
     |████████████████████████████████| 157 kB 56.9 MB/s 
     |████████████████████████████████| 157 kB 53.3 MB/s 
     |████████████████████████████████| 157 kB 54.5 MB/s 
     |████████████████████████████████| 157 kB 46.8 MB/s 
     |████████████████████████████████| 157 kB 68.2 MB/s 
     |████████████████████████████████| 157 kB 43.5 MB/s 
     |████████████████████████████████| 157 kB 60.5 MB/s 
     |████████████████████████████████| 156 kB 67.4 MB/s 


In [ ]:
# Log in to your user (you must first enter the key from the settings with wandb.ai through the console)
wandb.login()
# initialize the project
wandb.init(project="pytorch-demo")
# save the grid parameters in wandb + ask to monitor the grid gradients
wandb.watch(model);

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Training loss:,█▅▅▂▅▇▆▃▃▃▂▅▄▁▅▂▂▂▇▂
Validation loss:,▂▆▅▅▃▃▂▄▆▃▅▁▃▆▅▃▂▂▃▅▅▂▂▁▁▅▄▃▂█▇▂▃▂▄▅▃▃▂▄
Training loss:,0.5772
Validation loss:,22.10991


In [ ]:
#Define our variables:
set_random_seed(200)
train_set = MyDataset(X_train, y_train) 
train_loader = torch.utils.data.DataLoader(train_set, shuffle=True, batch_size = 600)

test_set = MyDataset(X_test, y_test) 
test_loader = torch.utils.data.DataLoader(test_set, batch_size = 600) 

model = model_2
#model_2  
# optimizer = torch.optim.SGD(model.parameters(), lr=0.0005)   
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999))
criterion = nn.MSELoss()

In [ ]:
def train(model, optimizer, criterion, train_loader, test_loader):
    '''
    params:
        model - torch.nn.Module to be fitted
        optimizer - model optimizer
        criterion - loss function from torch.nn
        train_loader - torch.utils.data.Dataloader with train set
        test_loader - torch.utils.data.Dataloader with test set
                      (if you wish to validate during training)
    '''
    # YOUR CODE HERE
    for epoch in range(14):
      for i in train_loader:
          train_x = i['sample']
          train_y = i['target']
          train_x = train_x.type(torch.FloatTensor)
          train_y = train_y.type(torch.FloatTensor).view(-1,1) 
          y_pred = model(train_x)
          loss = criterion(y_pred, train_y)
          loss.backward()
          optimizer.step()
          optimizer.zero_grad()                         
      wandb.log({"Training loss: ": loss.item()})

      #validation
      if epoch % 2 == 0:
              val_loss = [] # We will add average loss here
              model = model.eval() 
              with torch.no_grad():  # On validation we forbid the framework to read gradients by parameters
                for i in test_loader:
                  test_x = i['sample']
                  test_y = i['target']
                  test_x = test_x.type(torch.FloatTensor)
                  test_y = test_y.type(torch.FloatTensor).view(-1,1) 
                  y_pred_1 = model(test_x)
                  y_pred_1 = normalise.inverse_transform(y_pred_1.reshape(-1, 1))
                  test_y = normalise.inverse_transform(test_y)
                  loss = criterion(torch.from_numpy(y_pred_1), torch.from_numpy(test_y))  # count the loss
                  val_loss.append(loss.numpy())
                  wandb.log({"Validation loss: ": loss.item()})
              print(f"Epoch: {epoch}, loss: {(np.mean(val_loss))**(1/2)}") 
    

In [ ]:
train(model, optimizer, criterion, train_loader, test_loader)

Epoch: 0, loss: 9.13646585001669
Epoch: 2, loss: 8.965450600818663
Epoch: 4, loss: 8.939092874186292
Epoch: 6, loss: 8.911020808984798
Epoch: 8, loss: 8.89928623796139
Epoch: 10, loss: 8.869543516544788
Epoch: 12, loss: 8.863429254526485


In [ ]:
def test(model, criterion, test_loader):
    '''
    params:
        model - torch.nn.Module to be evaluated on test set
        criterion - loss function from torch.nn
        test_loader - torch.utils.data.Dataloader with test set
    ----------
    returns:
        predicts - torch.tensor with shape (len(test_loader.dataset), ),
                   which contains predictions for test objects
    '''
    # YOUR CODE HERE
    rmse_loss = []
    model=model.eval()                             
    with torch.no_grad(): 
        for i in test_loader:
            test_x = i['sample']
            test_y = i['target']
            test_x = test_x.type(torch.FloatTensor)
            test_y = test_y.type(torch.FloatTensor).view(-1,1) 
            y_pred_1 = model(test_x) # making predictions
            y_pred_1 = normalise.inverse_transform(y_pred_1.reshape(-1, 1))
            test_y = normalise.inverse_transform(test_y)
            loss = criterion(torch.from_numpy(y_pred_1), torch.from_numpy(test_y)) # count the loss
            rmse_loss.append(loss.numpy()) 
    predicts = torch.ones(len(test_loader.dataset))
    print((np.mean(rmse_loss))**(1/2))
    return predicts

In [ ]:
test(model, criterion, test_loader)

8.887600772227886


tensor([1., 1., 1.,  ..., 1., 1., 1.])

In [ ]:
assert test(model, criterion, test_loader).shape[0] == y_test.shape[0]

8.887600772227886


In [ ]:
######### REPORT!!!!! #########
#link to the wandb report: https://wandb.ai/livanovskaya/pytorch-demo/reports/-homework_1-9---VmlldzoyNzM5MDQy?accessToken=ditp05qoh0y57s0tgrz00e38qr3ahv8wlrmz671d7hyrv07r9a8t5t14nkiaijtf